In [1]:
! pip install huggingface_hub 

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 190.3/190.3 KB 15.5 MB/s eta 0:00:00


In [2]:
! pip install datasets

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 462.8/462.8 KB 36.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 213.0/213.0 KB 25.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.0/132.0 KB 16.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 140.6/140.6 KB 18.1 MB/s eta 0:00:00
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling urllib3-1.24.3:
      Successfully uninstalled urllib3-1.24.3


In [3]:
! pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.3/6.3 MB 60.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 97.2 MB/s eta 0:00:00


# hugging face hub 

In [4]:
from huggingface_hub import notebook_login

In [5]:
notebook_login()

Token is valid.
Your token has been saved in your configured git credential helpers (store).
Your token has been saved to /root/.cache/huggingface/token
Login successful


# import 

In [6]:
from transformers import AutoModelForSeq2SeqLM,Seq2SeqTrainingArguments,Seq2SeqTrainer,AutoTokenizer,DataCollatorForSeq2Seq,Conversation
from datasets import concatenate_datasets, load_dataset

# model and tokenizer

In [7]:
model = AutoModelForSeq2SeqLM.from_pretrained("t5-small")
tokenizer = AutoTokenizer.from_pretrained('t5-small')

/usr/local/lib/python3.8/dist-packages/transformers/models/t5/tokenization_t5_fast.py:155: FutureWarning: This tokenizer was incorrectly instantiated with a model max length of 512 which will be corrected in Transformers v5.
For now, this behavior is kept to avoid breaking backwards compatibility when padding/encoding with `truncation is True`.
- Be aware that you SHOULD NOT rely on t5-small automatically truncating your input to 512 when padding/encoding.
- If you want to encode/pad to sequences longer than 512 you can either instantiate this tokenizer with `model_max_length` or pass `max_length` when encoding/padding.
- To avoid this warning, please instantiate this tokenizer with `model_max_length` set to your preferred value.
  warnings.warn(


# process data

In [8]:
from datasets import concatenate_datasets, load_dataset

In [9]:
data = load_dataset("conv_ai_3")

Generating train split:   0%|          | 0/9176 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/2313 [00:00<?, ? examples/s]

Dataset conv_ai_3 downloaded and prepared to /root/.cache/huggingface/datasets/conv_ai_3/conv_ai_3/1.0.0/fee8db8bfc2268ecf5e609156cb7bdafa364dfca7d706ff808a2e71e8a006f64. Subsequent calls will reuse this data.


  0%|          | 0/2 [00:00<?, ?it/s]

In [10]:
def cleanr (data):
  from datasets import Dataset
  que = tokenizer(data["question"],padding=True,truncation=True)
  que = Dataset.from_dict(que)
  answ = tokenizer(data["answer"],padding=True,truncation=True)
  answ = Dataset.from_dict(answ)
  answ = answ.rename_column("input_ids", "labels")
  answ = answ.remove_columns(["attention_mask"])
  return concatenate_datasets([que, answ],axis=1)

In [11]:
train = cleanr(data["train"])
validate = cleanr(data["validation"])

# trainer

In [24]:
args = Seq2SeqTrainingArguments(
    'test',
    per_device_train_batch_size=128,
    per_device_eval_batch_size=128,
    learning_rate=2e-3,
    weight_decay=0.01,
    evaluation_strategy="epoch",
    logging_strategy="epoch",
    log_level="error",
    predict_with_generate=True,
    num_train_epochs=50,
    push_to_hub = True,
    # hub_model_id='organization/',
    # push_to_hub_token=token_login
)
collator = DataCollatorForSeq2Seq(tokenizer, model = model)

In [ ]:
args.

In [25]:
train = Seq2SeqTrainer(
    model,
    args,
    train_dataset = train,
    eval_dataset = validate,
    data_collator = collator,
    tokenizer = tokenizer
)


Cloning https://huggingface.co/Ahmade/test into local empty directory.


In [26]:
train.train()

/usr/local/lib/python3.8/dist-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


{'loss': 1.0036, 'learning_rate': 0.00196, 'epoch': 1.0}
{'eval_loss': 0.839471697807312, 'eval_runtime': 2.811, 'eval_samples_per_second': 822.852, 'eval_steps_per_second': 6.759, 'epoch': 1.0}
{'loss': 0.5944, 'learning_rate': 0.00192, 'epoch': 2.0}
{'eval_loss': 0.8436023592948914, 'eval_runtime': 2.7561, 'eval_samples_per_second': 839.223, 'eval_steps_per_second': 6.894, 'epoch': 2.0}
{'loss': 0.518, 'learning_rate': 0.00188, 'epoch': 3.0}
{'eval_loss': 0.8627429008483887, 'eval_runtime': 2.8027, 'eval_samples_per_second': 825.277, 'eval_steps_per_second': 6.779, 'epoch': 3.0}
{'loss': 0.4623, 'learning_rate': 0.00184, 'epoch': 4.0}
{'eval_loss': 0.8937391042709351, 'eval_runtime': 2.8732, 'eval_samples_per_second': 805.02, 'eval_steps_per_second': 6.613, 'epoch': 4.0}
{'loss': 0.4208, 'learning_rate': 0.0018000000000000002, 'epoch': 5.0}
{'eval_loss': 0.9090012907981873, 'eval_runtime': 2.9077, 'eval_samples_per_second': 795.463, 'eval_steps_per_second': 6.534, 'epoch': 5.0}
{'los

TrainOutput(global_step=3600, training_loss=0.19985971914397346, metrics={'train_runtime': 2667.5307, 'train_samples_per_second': 171.994, 'train_steps_per_second': 1.35, 'train_loss': 0.19985971914397346, 'epoch': 50.0})

In [27]:
train.push_to_hub("hello ")

Upload file pytorch_model.bin:   0%|          | 32.0k/231M [00:00<?, ?B/s]

Upload file runs/Feb20_11-41-47_031bf9734885/events.out.tfevents.1676893328.031bf9734885.705.0: 100%|#########…

remote: Scanning LFS files for validity...        
remote: LFS file scan complete.        
To https://huggingface.co/Ahmade/test
   0d0c3cf..fb08bd6  main -> main

remote: LFS file scan complete.        
To https://huggingface.co/Ahmade/test
   0d0c3cf..fb08bd6  main -> main

To https://huggingface.co/Ahmade/test
   fb08bd6..de813ae  main -> main

   fb08bd6..de813ae  main -> main



'https://huggingface.co/Ahmade/test/commit/fb08bd663341b634bb51ee76b02e10b1ca8ae250'

In [ ]:
model.push_to_hub('test')
tokenizer.push_to_hub("test")

In [15]:
tokenizer("hello word")

{'input_ids': [21820, 1448, 1], 'attention_mask': [1, 1, 1]}

In [19]:
pred =  train.predict(validate)

# chatbot

In [18]:
from huggingface_hub import notebook_login
notebook_login()

In [ ]:
def makeChatbot(model_name,data_name,name,token_login,commit=""):
  from transformers import AutoModelForSeq2SeqLM,Seq2SeqTrainingArguments,Seq2SeqTrainer,AutoTokenizer,DataCollatorForSeq2Seq,Conversation
  from datasets import concatenate_datasets, load_dataset,Dataset,load_metric
  model = AutoModelForSeq2SeqLM.from_pretrained(model_name)
  tokenizer = AutoTokenizer.from_pretrained(model_name)
  data = load_dataset(data_name)
  metric = load_metric(data_name)
  collator = DataCollatorForSeq2Seq(tokenizer, model = model)
  def cleanr (data):
    que = tokenizer(data["question"],padding=True,truncation=True)
    que = Dataset.from_dict(que)
    answ = tokenizer(data["answer"],padding=True,truncation=True)
    answ = Dataset.from_dict(answ)
    answ = answ.rename_column("input_ids", "labels")
    answ = answ.remove_columns(["attention_mask"])
    return concatenate_datasets([que, answ],axis=1)
  def compute_metrics(eval_pred):
      logits , labels = eval_pred
      predictions = np.argmax(logits,axis=1)
      return metric.compute(predictions=predictions,references=labels)

  train = cleanr(data["train"])
  validate = cleanr(data["validation"])
  # test = cleanr(data["test"])
  args = Seq2SeqTrainingArguments(
    name,
    per_device_train_batch_size=128,
    per_device_eval_batch_size=128,
    learning_rate=2e-3,
    weight_decay=0.01,
    evaluation_strategy="epoch",
    logging_strategy="epoch",
    log_level="error",
    predict_with_generate=True,
    num_train_epochs=50,
    push_to_hub = True,
  )
  train = Seq2SeqTrainer(
    model,
    args,
    train_dataset = train,
    eval_dataset = validate,
    data_collator = collator,
    tokenizer = tokenizer,
    compute_metrics = compute_metrics
  )
  train.train()
  train.push_to_hub(commit)
  model.push_to_hub(name)
tokenizer.push_to_hub(name)